## AI Pipeline

In [3]:
import json
import numpy as np
import pandas as pd
import os
import glob
import shutil

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

current_directory = os.getcwd()
print(current_directory)

source_directory = '../data'
file_directory = '../data/sample'

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)

/Users/jleelong/workspace/sample-similarity-search/analysis


## Preprocessing

In [4]:
def remove_na_attributes(data):
    """
    Recursively removes attributes with NA values from a JSON-like dictionary.

    Args:
        data (dict): A dictionary representing JSON data.

    Returns:
        dict: A new dictionary with NA attributes removed.
    """
    if isinstance(data, dict):
        return {
            k: remove_na_attributes(v) for k, v in data.items() if v is not None and v != "NA"
        }
    elif isinstance(data, list):
        return [remove_na_attributes(item) for item in data]
    else:
        return data


def load_json_files_to_dataframe(file_dir):
    """
    Loads multiple JSON files from a directory into a single Pandas DataFrame.

    Args:
        file_dir (str): The directory containing the JSON files.

    Returns:
        pd.DataFrame: A DataFrame containing the combined data from all JSON files.
                      Returns an empty DataFrame if no files are found or if an error occurs.
    """
    all_files = os.listdir(file_dir)
    json_files = [f for f in all_files if f.endswith('.json')]
    
    if not json_files:
        print(f"No JSON files found in directory: {file_dir}")
        return pd.DataFrame()

    all_data = []
    for file_name in json_files:
        file_path = os.path.join(file_dir, file_name)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                all_data.append(data)
        except json.JSONDecodeError:
            print(f"Error decoding JSON in file: {file_path}")
        except FileNotFoundError:
             print(f"File not found: {file_path}")

    if not all_data:
        return pd.DataFrame()
    
    if not isinstance(all_data, list):
         return pd.DataFrame()

    # all_data = remove_na_attributes(all_data)
    return pd.concat([pd.json_normalize(data) for data in all_data], ignore_index=True)

In [5]:
def convert_building_class(value):
    if pd.notna(value):
        if value.isdigit():
            return int(value)
        else:
            try:
                value = value.replace('+', '.5')
                return round(float(value))
            except ValueError:
                return None

def fill_building_class(tot_asmt, bld_cls, avg_bld_class):
    if (pd.isna(bld_cls)):
        valid_indices = avg_bld_class.index[avg_bld_class.index <= tot_asmt]
        if not valid_indices.empty:
          return avg_bld_class[valid_indices.max()]
        valid_indices = avg_bld_class.index[avg_bld_class.index > tot_asmt]
        if not valid_indices.empty:
          return avg_bld_class[valid_indices.min()]
        else:
          return 0
    else:
        return bld_cls

def fill_square_feet(bld_asmt, sq_ft, avg_feet):
    if (pd.isna(sq_ft)):
        valid_indices = avg_feet.index[avg_feet.index <= bld_asmt]
        if not valid_indices.empty:
          return avg_feet[valid_indices.max()]
        valid_indices = avg_feet.index[avg_feet.index > bld_asmt]
        if not valid_indices.empty:
          return avg_feet[valid_indices.min()]
        else:
          return 0
    else:
        return sq_ft

def fill_no_of_dwellings(bld_cls, sq_ft, no_dwell, avg_dwellings):
    if np.isnan(no_dwell):
        result = 0
        segment = avg_dwellings[(avg_dwellings['Building_Class'] == bld_cls)]     
        if not segment.empty:
            filtered = segment[segment['Sq_Ft'] <= sq_ft]
            if not filtered.empty:
                result = filtered.loc[filtered.idxmax()['Sq_Ft']]['No_Of_Dwellings']
            else:
                filtered = segment[segment['Sq_Ft'] > sq_ft]
                if not filtered.empty:
                    result = filtered.loc[filtered.idxmin()['Sq_Ft']]['No_Of_Dwellings']
        return round(result)
    else:
        return no_dwell

def fill_year_built(bld_cls, tax_rate, sq_ft, yr_built, avg_year, avg_year_class):
    if np.isnan(yr_built):
        result = 0
        segment = avg_year[(avg_year['Building_Class'] == bld_cls) &
                           (avg_year['TaxRate'] == tax_rate)]
        if segment.empty:
            segment = avg_year_class[avg_year_class['Building_Class'] == bld_cls]        
        if not segment.empty:
            filtered = segment[segment['Sq_Ft'] <= sq_ft]
            if not filtered.empty:
                result = filtered.loc[filtered.idxmax()['Sq_Ft']]['Yr_Built']
            else:
                filtered = segment[segment['Sq_Ft'] > sq_ft]
                if not filtered.empty:
                    result = filtered.loc[filtered.idxmin()['Sq_Ft']]['Yr_Built']
        return round(result)
    else:
        return yr_built

def fill_recorded_taxes(calc_tax, sq_ft, rec_tax, avg_taxes):
    if np.isnan(rec_tax):
        result = calc_tax
        segment = avg_taxes[avg_taxes['Calculated_Taxes'] == calc_tax]
        if segment.empty:
            filtered = avg_taxes[avg_taxes['Calculated_Taxes'] <= calc_tax]
            if not filtered.empty:
                calc_tax = filtered.loc[filtered.idxmax()['Calculated_Taxes']]['Calculated_Taxes']
                segment = avg_taxes[avg_taxes['Calculated_Taxes'] == calc_tax]
            else:
                filtered = avg_taxes[avg_taxes['Calculated_Taxes'] > calc_tax]
                if not filtered.empty:
                    calc_tax = filtered.loc[filtered.idxmin()['Calculated_Taxes']]['Calculated_Taxes']
                    segment = avg_taxes[avg_taxes['Calculated_Taxes'] == calc_tax]    
        if not segment.empty:
            filtered = segment[segment['Sq_Ft'] <= sq_ft]
            if not filtered.empty:
                result = filtered.loc[filtered.idxmax()['Sq_Ft']]['Recorded_Taxes']
            else:
                filtered = segment[segment['Sq_Ft'] > sq_ft]
                if not filtered.empty:
                    result = filtered.loc[filtered.idxmin()['Sq_Ft']]['Recorded_Taxes']
        return result
    else:
        return rec_tax

def fill_corporate_owned(absentee, no_dwell, bld_cls, corp_owned,
                         avg_corp_owned, avg_corp_owned_dwell, avg_corp_owned_absent):
    if pd.isnull(corp_owned):
        result = 0
        segment = avg_corp_owned[(avg_corp_owned['Absentee'] == absentee) &
                                 (avg_corp_owned['No_Of_Dwellings'] == no_dwell) &
                                 (avg_corp_owned['Building_Class'] == bld_cls)]
        if segment.empty:
            segment = avg_corp_owned_dwell[(avg_corp_owned_dwell['Absentee'] == absentee) &
                                           (avg_corp_owned_dwell['No_Of_Dwellings'] == no_dwell)]
        if segment.empty:
            segment = avg_corp_owned_absent[(avg_corp_owned_absent['Absentee'] == absentee)]  
        if not segment.empty:
            result = segment.mean()['Corporate_Owned']
        return round(result)
    else:
        return corp_owned

In [6]:
def preprocess_data(file_dir, model_name):
    df = load_json_files_to_dataframe(file_dir)
    df = df.dropna(subset=['countyData.Sale_Price'])
    
    columns_to_keep = ['location.lat', 'location.lng', 'countyData.No_Of_Dwellings',
                       'countyData.Corporate_Owned', 'countyData.Absentee', 'countyData.NU_Code',
                       'countyData.updated_at', 'countyData.TotalUnits', 'countyData.Sq_Ft',
                       'countyData.Property_Class', 'countyData.Building_Class', 'countyData.Yr_Built',
                       'countyData.Sale_Date', 'countyData.TaxRate', 'countyData.TaxRatio',
                       'countyData.RateYear', 'countyData.Recorded_Taxes', 'countyData.Calculated_Taxes',
                       'countyData.Calculated_Taxes_Year', 'countyData.Year_1', 'countyData.Land_Assmnt_1',
                       'countyData.Building_Assmnt_1', 'countyData.Total_Assmnt_1', 'countyData.Sale_Price']
    df = df[columns_to_keep]
    df = df.rename(columns={'location.lat': 'lat', 'location.lng': 'lng',
                            'countyData.No_Of_Dwellings': 'No_Of_Dwellings',
                            'countyData.Corporate_Owned': 'Corporate_Owned',
                            'countyData.Absentee': 'Absentee', 'countyData.NU_Code': 'NU_Code',
                            'countyData.updated_at': 'updated_at', 'countyData.TotalUnits': 'TotalUnits',
                            'countyData.Sq_Ft': 'Sq_Ft', 'countyData.Property_Class': 'Property_Class',
                            'countyData.Building_Class': 'Building_Class', 'countyData.Yr_Built': 'Yr_Built',
                            'countyData.Sale_Date': 'Sale_Date', 'countyData.TaxRate': 'TaxRate',
                            'countyData.TaxRatio': 'TaxRatio', 'countyData.RateYear': 'RateYear',
                            'countyData.Recorded_Taxes': 'Recorded_Taxes',
                            'countyData.Calculated_Taxes': 'Calculated_Taxes',
                            'countyData.Calculated_Taxes_Year': 'Calculated_Taxes_Year',
                            'countyData.Year_1': 'Year_1', 'countyData.Land_Assmnt_1': 'Land_Assmnt_1',
                            'countyData.Building_Assmnt_1': 'Building_Assmnt_1',
                            'countyData.Total_Assmnt_1': 'Total_Assmnt_1', 'countyData.Sale_Price': 'Sale_Price'})
    
    df = df.dropna(subset=['Land_Assmnt_1', 'Building_Assmnt_1', 'Total_Assmnt_1'])
    df = df.dropna(subset=['lat', 'lng'])
    
#     df['County'] = df['id'].apply(lambda x: int(x[:2].lstrip("0")))
#     df['Municipality'] = df['id'].apply(lambda x: int(x[2:4].lstrip("0")))
#     df = df.drop('id', axis=1)
    
    df['NU_Code'] = df['NU_Code'].apply(lambda x: int(x) if pd.notna(x) and x.isdigit()
                                        else -1 if pd.notna(x) else 99)
    df = df[df['NU_Code'] == 99]
    df = df.drop('NU_Code', axis=1)
    
    df['updated_at'] = pd.to_datetime(df['updated_at']).values.astype('datetime64[ms]').astype(int)
    df['updated_at'] = df['updated_at'].fillna(0)
    
    df['Property_Class'] = df['Property_Class'].map({
        '1': 1, '2': 2, '3A': 3, '3B': 4, '4A': 5, '4B': 6, '4C': 7, '5A': 8, '5B': 9, '6A': 10,
        '6B': 11, '6C': 12, '15A': 13, '15B': 14, '15C': 15, '15D': 16, '15E': 17, '15F': 18
    })
    df = df[df['Property_Class'] == 2]
    df = df.drop(['Property_Class', 'TotalUnits'], axis=1)
    
    df['Sale_Date'] = pd.to_datetime(df['Sale_Date'])
    df['Sale_Month'] = df['Sale_Date'].dt.month.astype(pd.Int64Dtype())
    df['Sale_Year'] = df['Sale_Date'].dt.year.astype(pd.Int64Dtype())
    df = df.drop('Sale_Date', axis=1)
    df = df.dropna(subset=['Sale_Month', 'Sale_Year'])

    df['TaxRate'] = df['TaxRate'].astype(float)
    df['TaxRatio'] = df['TaxRatio'].astype(float)
    df['RateYear'] = df['RateYear'].astype(int)

    df['Building_Class'] = df['Building_Class'].apply(lambda x: convert_building_class(x)).astype(pd.Int64Dtype())
    update_mapping = pd.DataFrame({
        'key':[
            26, 27, 28, 29, 30,
            33, 34, 35, 36, 37, 38, 39,
            43, 44, 45, 46, 47, 48, 49,
            50, 51, 52, 53, 54, 55
        ], 'value':[
            12, 13, 14, 16, 18,
            13, 14, 15, 16, 17, 18, 19,
            13, 14, 15, 16, 17, 18, 19,
            12, 13, 15, 18, 19, 20
        ]
    })
    df['Building_Class'] = df['Building_Class'].replace(dict(zip(update_mapping['key'], update_mapping['value'])))
    avg_bld_class = df[df['Building_Class'].notna()].groupby('Total_Assmnt_1').mean()['Building_Class']
    df['Building_Class'] = df.apply(lambda x: fill_building_class(x['Total_Assmnt_1'],
                                                                  x['Building_Class'], avg_bld_class), axis=1)
    df['Building_Class'] = df['Building_Class'].astype(pd.Float64Dtype()).round().astype(pd.Int64Dtype())
        
    avg_feet = df[df['Sq_Ft'].notna()].groupby('Building_Assmnt_1').mean()['Sq_Ft']
    df['Sq_Ft'] = df.apply(lambda x: fill_square_feet(x['Building_Assmnt_1'], x['Sq_Ft'], avg_feet), axis=1)

    avg_dwellings = df[df['No_Of_Dwellings'].notna()].groupby(['Building_Class', 'Sq_Ft']).agg(
        {'No_Of_Dwellings': 'mean'}).reset_index()
    df['No_Of_Dwellings'] = df.apply(lambda x: fill_no_of_dwellings(x['Building_Class'],
                                                                    x['Sq_Ft'], x['No_Of_Dwellings'],
                                                                    avg_dwellings), axis=1)
    df['No_Of_Dwellings'] = df['No_Of_Dwellings'].astype(int)

    avg_year = df[df['Yr_Built'].notna()].groupby(['Building_Class', 'TaxRate', 'Sq_Ft']).agg(
        {'Yr_Built': 'mean'}).reset_index()
    avg_year_class = df[df['Yr_Built'].notna()].groupby(['Building_Class', 'Sq_Ft']).agg(
        {'Yr_Built': 'mean'}).reset_index()
    df['Yr_Built'] = df.apply(lambda x: fill_year_built(x['Building_Class'], x['TaxRate'],
                                                        x['Sq_Ft'], x['Yr_Built'],
                                                        avg_year, avg_year_class), axis=1)
    df['Yr_Built'] = df['Yr_Built'].astype(int)

    avg_taxes = df[df['Recorded_Taxes'].notna()].groupby(['Calculated_Taxes', 'Sq_Ft']).agg(
        {'Recorded_Taxes': 'mean'}).reset_index()
    df['Recorded_Taxes'] = df.apply(lambda x: fill_recorded_taxes(x['Calculated_Taxes'], x['Sq_Ft'],
                                                                  x['Recorded_Taxes'], avg_taxes), axis=1)
    
    df = df.drop(['Calculated_Taxes', 'Calculated_Taxes_Year'], axis=1)

    avg_corp_owned = df[df['Corporate_Owned'].notna()].groupby(['Absentee', 'No_Of_Dwellings',
                                                                'Building_Class']).agg(
        {'Corporate_Owned': 'mean'}).reset_index()
    avg_corp_owned_dwell = df[df['Corporate_Owned'].notna()].groupby(['Absentee', 'No_Of_Dwellings']).agg(
        {'Corporate_Owned': 'mean'}).reset_index()
    avg_corp_owned_absent = df[df['Corporate_Owned'].notna()].groupby(['Absentee']).agg(
        {'Corporate_Owned': 'mean'}).reset_index()
    df['Corporate_Owned'] = df.apply(lambda x: fill_corporate_owned(x['Absentee'], x['No_Of_Dwellings'],
                                                                    x['Building_Class'], x['Corporate_Owned'],
                                                                    avg_corp_owned, avg_corp_owned_dwell,
                                                                    avg_corp_owned_absent), axis=1)

#     df['County'] = df['County'].map({
#         1: 'Atlantic',
#         2: 'Bergen',
#         3: 'Burlignton',
#         4: 'Camden',
#         5: 'Cape May',
#         6: 'Cumberland',
#         7: 'Essex',
#         8: 'Gloucester',
#         9: 'Hudson',
#         10: 'Hunterdon',
#         11: 'Mercer',
#         12: 'Middlesex',
#         13: 'Monmouth',
#         14: 'Morris',
#         15: 'Ocean',
#         16: 'Passaic',
#         17: 'Salem',
#         18: 'Somerset',
#         19: 'Sussex',
#         20: 'Union',
#         21: 'Warren'
#     })
#     dummies = pd.get_dummies(df['County'])
#     df = df.drop('County', axis=1)
#     # in case sample data is missing a county, we'll add
#     # the columns manually and then overwrite the values
#     # skip Atlantic because we want to drop one column
#     counties = ['Bergen', 'Burlignton', 'Camden', 'Cape May',
#                 'Cumberland', 'Essex', 'Gloucester', 'Hudson',
#                 'Hunterdon', 'Mercer', 'Middlesex', 'Monmouth',
#                 'Morris', 'Ocean', 'Passaic', 'Salem', 'Somerset',
#                 'Sussex', 'Union', 'Warren']
#     for county in counties:
#         if county in dummies.columns:
#             df[county] = dummies[county]
#         else:
#             df[county] = False
    
    df['Year_1'] = df['Year_1'].astype(int)
    df.to_pickle(os.path.join(file_dir, 'processed', model_name + ".pkl"))

## Train the Model

In [7]:
def scale_and_fit(file_dir, model_name, X_train, X_val, y_train, y_val):
    file_path = os.path.join(file_dir, 'scaler', model_name + ".save")
    if os.path.exists(file_path):
        scaler = joblib.load(file_path)
        X_train = scaler.transform(X_train)
    else:
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        joblib.dump(scaler, file_path)

    X_val = scaler.transform(X_val)
#     X_test = scaler.transform(X_test)
    
    file_path = os.path.join(file_dir, 'model', model_name + ".h5")
    if os.path.exists(file_path):
        model = load_model(file_path)
    else:
        model = Sequential()
        model.add(Dense(19, activation='relu')) # Inputs

        model.add(Dense(304, activation='relu')) # First hidden layer
        model.add(Dropout(0.2))
        # model.add(BatchNormalization())

        model.add(Dense(304, activation='relu')) # Second hidden layer
        model.add(Dropout(0.2))
        # model.add(BatchNormalization())

        model.add(Dense(304, activation='relu')) # Third hidden layer
        model.add(Dropout(0.2))
        # model.add(BatchNormalization())

        model.add(Dense(304, activation='relu')) # Forth hidden layer
        model.add(Dropout(0.2))
        # model.add(BatchNormalization())

        model.add(Dense(1, activation='relu')) # Output layer (regression)
        model.compile(optimizer='adam', loss='mse')
    
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
    model.fit(x=X_train, y=y_train, epochs=1000, batch_size=32,
              validation_data=(X_val, y_val), verbose=0, callbacks=[early_stop])
    model.save(file_path)

def train_the_model(file_dir, model_name):
    file_path = os.path.join(file_dir, 'processed', model_name + ".pkl")
    if os.path.exists(file_path):
        # shuffle the data
        df = pd.read_pickle(file_path).sample(frac=1, random_state=42).reset_index(drop=True)
    else:
        raise ValueError(f"{file_path} does not exist")
    
    X = df.drop('Sale_Price', axis=1).values
    y = df['Sale_Price'].values

    # perform a 60% / 20% / 20% split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    scale_and_fit(file_dir, model_name, X_train, X_test, y_train, y_test)
    scale_and_fit(file_dir, 'all-county', X_train, X_test, y_train, y_test)


## Execute

In [8]:
# !pip install ipynb
# !pip install ipywidgets
# !pip install ipycanvas
# !pip install ipyevents
# !pip install watermark

In [9]:
%load_ext watermark
%watermark -p ipywidgets,ipycanvas,ipyevents

ipywidgets: 8.1.6
ipycanvas : 0.13.3
ipyevents : 2.0.2



In [10]:
import ipywidgets as widgets
import ipycanvas
from IPython.display import display, HTML

In [11]:
import time
import threading

pause_button = widgets.Button(
    description = 'pause',
    disabled = False,
    button_style = 'warning',
    tooltip = 'pause training the model',
    icon = ''
)

continue_button = widgets.Button(
    description = 'continue',
    disabled = False,
    button_style = 'success',
    tooltip = 'continue training the model',
    icon = ''
)

file_path = ''
paused = True
started = False
counter = 0

output = widgets.Output()
def refreshOutput():
    global counter
    with output:
        output.clear_output()
        if not started:
            display(HTML("<font size=\"5\" color=\"blue\">not started</font><br/>"))
        elif file_path == None:
            display(HTML("<font size=\"5\" color=\"orange\">DONE</font><br/>"))
        elif paused:
            display(HTML("<font size=\"5\" color=\"red\">paused!!</font><br/>"))
        else:
            display(HTML(f"<font size=\"5\" color=\"green\">loading {file_path}...</font><br/>"))
        display(HTML(f"<font size=\"5\" color=\"purple\">counter at {counter}</font><br/>"))

header = widgets.Output()
with header:
    display(HTML("<h1>Let's Train the Model</h1?"))
    display(HTML("<font size=\"3\">hit pause or continue to control the iterations</font><br/>"))
    refreshOutput()

def remove_files_matching(directory, extension):
    files = glob.glob(os.path.join(directory, f"*.{extension}"))
    for file_path in files:
        os.remove(file_path)

def pause_processing(button):
    global paused
    paused = True
    refreshOutput()
    
def continue_processing(button):
    global paused, counter
    paused = False
    if not thr.is_alive():
        counter = 0
        thr.start()
    refreshOutput()

pause_button.on_click(pause_processing)
continue_button.on_click(continue_processing)

def load():
    global file_path, counter
    
    all_files = os.listdir(source_directory)
    json_files = [f for f in all_files if f.endswith('.json')]
    if not json_files:
        raise ValueError(f"No JSON files found in directory: {source_directory}")
    
    for file_name in json_files:
        while paused:
            time.sleep(1)
        model_name = os.path.splitext(file_name)[0]
        file_path = os.path.join(file_directory, 'model', model_name + ".h5")
        if os.path.exists(file_path):
            continue
        file_path = os.path.join(source_directory, file_name)
        refreshOutput()
        shutil.copy(file_path, file_directory)
        preprocess_data(file_directory, model_name)
        train_the_model(file_directory, model_name)
        remove_files_matching(file_directory, "json")
        counter += 1
    file_path = None
    refreshOutput()

thr = threading.Thread(target=load)
started = True

### USER INTERFACE
header_box = widgets.HBox([header])
button_box = widgets.HBox([pause_button, continue_button])
output_box = widgets.HBox([output])
widgets.VBox([header_box, button_box, output])